# Introduction to XGBoost Spark with GPU

The goal of this notebook is to show how to train a XGBoost Model with Spark RAPIDS XGBoost library on GPUs. The dataset used with this notebook is derived from Fannie Mae’s Single-Family Loan Performance Data with all rights reserved by Fannie Mae. This processed dataset is redistributed with permission and consent from Fannie Mae. This notebook uses XGBoost to train 12-month mortgage loan delinquency prediction model .

A few libraries required for this notebook:
  1. NumPy
  2. cudf jar
  3. xgboost4j jar
  4. xgboost4j-spark jar
  5. rapids-4-spark.jar

This notebook also illustrates the ease of porting a sample CPU based Spark xgboost4j code into GPU. There is only one change required for running Spark XGBoost on GPU. That is replacing the API `setFeaturesCol(feature)` on CPU with the new API `setFeaturesCols(features)`. This also eliminates the need for vectorization (assembling multiple feature columns in to one column) since we can read multiple columns.

In [1]:
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark-3.2.1-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--files /opt/spark-3.2.1-bin-hadoop3.2/examples/src/main/scripts/getGpusResources.sh --conf spark.plugins=com.nvidia.spark.SQLPlugin --jars /opt/xgboost/cudf-22.02.0-cuda11.jar,/opt/xgboost/rapids-4-spark_2.12-22.02.0.jar,/opt/xgboost/xgboost4j_3.0-1.4.2-0.2.0.jar,/opt/xgboost/xgboost4j-spark_3.0-1.4.2-0.2.0.jar pyspark-shell'


import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.addPyFile('/opt/xgboost/xgboost4j-spark_3.0-1.4.2-0.2.0.jar')

/opt/spark-3.2.1-bin-hadoop3.2/python/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


#### Import All Libraries

In [2]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType
from time import time
import os

Besides CPU version requires two extra libraries.
```Python
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
```

#### Create Spark Session and Data Reader

In [3]:
# Make sure it runs on GPU
spark.conf.set("spark.rapids.sql.enabled", "true")
spark.conf.set("spark.rapids.sql.incompatibleDateFormats.enabled","true")
spark.conf.set("spark.rapids.sql.csv.read.integer.enabled","true")
spark.conf.set("spark.rapids.sql.csv.read.long.enabled","true")
spark.conf.set("spark.rapids.sql.csv.read.double.enabled","true")
spark.conf.set("spark.rapids.sql.incompatibleDateFormats.enabled","true")

#spark.conf.set("spark.plugins=com.nvidia.spark.SQLPlugin")
spark.conf.set("spark.rapids.memory.gpu.pooling.enabled","false")
spark.conf.set("spark.executor.resource.gpu.amount","1")
spark.conf.set("spark.task.resource.gpu.amount","1")
spark.conf.set("spark.executor.resource.gpu.discoveryScript","/opt/spark-3.2.1-bin-hadoop3.2/examples/src/main/scripts/getGpusResources.sh")

reader = spark.read

#### Specify the Data Schema and Load the Data

In [8]:
label = 'delinquency_12'
schema = StructType([
    StructField('orig_channel', FloatType()),
    StructField('first_home_buyer', FloatType()),
    StructField('loan_purpose', FloatType()),
    StructField('property_type', FloatType()),
    StructField('occupancy_status', FloatType()),
    StructField('property_state', FloatType()),
    StructField('product_type', FloatType()),
    StructField('relocation_mortgage_indicator', FloatType()),
    StructField('seller_name', FloatType()),
    StructField('mod_flag', FloatType()),
    StructField('orig_interest_rate', FloatType()),
    StructField('orig_upb', IntegerType()),
    StructField('orig_loan_term', IntegerType()),
    StructField('orig_ltv', FloatType()),
    StructField('orig_cltv', FloatType()),
    StructField('num_borrowers', FloatType()),
    StructField('dti', FloatType()),
    StructField('borrower_credit_score', FloatType()),
    StructField('num_units', IntegerType()),
    StructField('zip', IntegerType()),
    StructField('mortgage_insurance_percent', FloatType()),
    StructField('current_loan_delinquency_status', IntegerType()),
    StructField('current_actual_upb', FloatType()),
    StructField('interest_rate', FloatType()),
    StructField('loan_age', FloatType()),
    StructField('msa', FloatType()),
    StructField('non_interest_bearing_upb', FloatType()),
    StructField(label, IntegerType()),
])
features = [ x.name for x in schema if x.name != label ]

# You need to update them to your real paths!
dataRoot = os.getenv("DATA_ROOT", "/opt/xgboost")
train_data = reader.schema(schema).option('header', True).csv(dataRoot + '/mortgage/csv/train')
trans_data = reader.schema(schema).option('header', True).csv(dataRoot + '/mortgage/csv/test')

Note on CPU version, vectorization is required before fitting data to classifier, which means you need to assemble all feature columns into one column.

```Python
def vectorize(data_frame):
    to_floats = [ col(x.name).cast(FloatType()) for x in data_frame.schema ]
    return (VectorAssembler()
        .setInputCols(features)
        .setOutputCol('features')
        .transform(data_frame.select(to_floats))
        .select(col('features'), col(label)))

train_data = vectorize(train_data)
trans_data = vectorize(trans_data)
```

#### Create a XGBoostClassifier

In [9]:
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 10, 
    'maxLeaves': 256,
    'objective':'binary:logistic',
    'growPolicy': 'depthwise',
    'minChildWeight': 30.0,
    'lambda_': 1.0,
    'scalePosWeight': 2.0,
    'subsample': 1.0,
    'nthread': 1,
    'numRound': 100,
    'numWorkers': 1,
}
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)

The CPU version classifier provides the API `setFeaturesCol` which only accepts a single column name, so vectorization for multiple feature columns is required.
```Python
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCol('features')
```

The parameter `num_workers` should be set to the number of GPUs in Spark cluster for GPU version, while for CPU version it is usually equal to the number of the CPU cores.

Concerning the tree method, GPU version only supports `gpu_hist` currently, while `hist` is designed and used here for CPU training.

#### Train the Data with Benchmark

In [10]:
def with_benchmark(phrase, action):
    start = time()
    result = action()
    end = time()
    print('{} takes {} seconds'.format(phrase, round(end - start, 2)))
    return result
model = with_benchmark('Training', lambda: classifier.fit(train_data))

Training takes 15.52 seconds


#### Save and Reload the Model

In [11]:
model.write().overwrite().save(dataRoot + '/new-model-path')
loaded_model = XGBoostClassificationModel().load(dataRoot + '/new-model-path')

#### Transformation and Show Result Sample

In [12]:
def transform():
    result = loaded_model.transform(trans_data).cache()
    result.foreachPartition(lambda _: None)
    return result
result = with_benchmark('Transformation', transform)
result.select(label, 'rawPrediction', 'probability', 'prediction').show(5)

Transformation takes 2.68 seconds
+--------------+--------------------+--------------------+----------+
|delinquency_12|       rawPrediction|         probability|prediction|
+--------------+--------------------+--------------------+----------+
|             0|[4.46620130538940...|[0.98863965645432...|       0.0|
|             0|[4.79745244979858...|[0.99181677773594...|       0.0|
|             0|[4.79745244979858...|[0.99181677773594...|       0.0|
|             0|[4.79745244979858...|[0.99181677773594...|       0.0|
|             0|[4.46620130538940...|[0.98863965645432...|       0.0|
+--------------+--------------------+--------------------+----------+
only showing top 5 rows



#### Evaluation

In [13]:
accuracy = with_benchmark(
    'Evaluation',
    lambda: MulticlassClassificationEvaluator().setLabelCol(label).evaluate(result))
print('Accuracy is ' + str(accuracy))

Evaluation takes 0.37 seconds
Accuracy is 1.0


In [14]:
spark.stop()